In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import auc,roc_curve
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("Shape of the training Data: ",train.shape)
print("Shape of test Data: ",test.shape)

In [ ]:
test.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
cols = test.columns

In [ ]:
X_train = train[cols]
X_test = test.copy()
y = train['claim']

In [ ]:
X_train = X_train.drop('id',axis=1)
X_test = X_test.drop('id',axis=1)

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

# XGB Classifier with KFold

In [ ]:
params_xgb = {'tree_method':'gpu_hist',
        'predictor':'gpu_predictor'}

In [ ]:
test_preds = np.zeros((X_test_scaled.shape[0],))

In [ ]:
test_preds.shape

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits,shuffle=True)

for jj,(train_index,val_index) in enumerate(kf.split(X_train)):
    train_features = X_train_scaled[train_index]
    train_target = y[train_index]
    
    val_features = X_train_scaled[val_index]
    val_target = y[val_index]
    
    model = XGBClassifier(**params_xgb)
    model.fit(train_features,train_target)
    
    test_preds += model.predict_proba(X_test_scaled)[:,1] / n_splits

In [ ]:
test_preds

In [ ]:
sub

In [ ]:
sub['claim'] = test_preds

In [ ]:
sub.to_csv('submission_kfold.csv',index=False)